# RiskShield: Supply Chain Risk Optimization

## Problem Background
RiskShield Electronics needs to design a resilient supply chain for critical electronic components. They must:
- Source from multiple suppliers to reduce risk
- Maintain buffer inventory for key components
- Plan contingency transportation routes
- Balance cost with risk mitigation

## Risk Factors
1. Supplier reliability (historical performance)
2. Geographic risk (natural disasters, political)
3. Transportation disruption probability
4. Demand uncertainty
5. Price volatility

## Optimization Goals
1. Minimize expected total cost
2. Maximize supply chain resilience
3. Ensure service level targets
4. Optimize buffer inventory levels
5. Balance supplier portfolio

In [ ]:
import pulp
import pandas as pd
import numpy as np

# Problem parameters
components = ['CPU', 'Memory', 'Storage']
suppliers = [f'Supplier_{i}' for i in range(1, 6)]  # 5 potential suppliers
regions = ['Asia', 'Europe', 'Americas']

# Supplier reliability scores (0-1)
reliability = {
    'Supplier_1': 0.95,
    'Supplier_2': 0.88,
    'Supplier_3': 0.92,
    'Supplier_4': 0.85,
    'Supplier_5': 0.90
}

# Component costs by supplier
cost = {
    'CPU': {'Supplier_1': 100, 'Supplier_2': 95, 'Supplier_3': 105, 'Supplier_4': 90, 'Supplier_5': 98},
    'Memory': {'Supplier_1': 45, 'Supplier_2': 48, 'Supplier_3': 42, 'Supplier_4': 50, 'Supplier_5': 44},
    'Storage': {'Supplier_1': 60, 'Supplier_2': 58, 'Supplier_3': 65, 'Supplier_4': 55, 'Supplier_5': 62}
}

# Geographic risk factors (higher = more risky)
geo_risk = {
    'Supplier_1': 0.15,  # Asia
    'Supplier_2': 0.10,  # Europe
    'Supplier_3': 0.08,  # Americas
    'Supplier_4': 0.12,  # Asia
    'Supplier_5': 0.09   # Europe
}

# Demand and service level requirements
demand = {
    'CPU': 1000,
    'Memory': 2000,
    'Storage': 1500
}

service_level = {
    'CPU': 0.99,
    'Memory': 0.95,
    'Storage': 0.97
}

# Create optimization model
prob = pulp.LpProblem("RiskShield_Optimization", pulp.LpMinimize)

# Decision variables
order = pulp.LpVariable.dicts("order",
    ((c, s) for c in components for s in suppliers),
    lowBound=0)

use_supplier = pulp.LpVariable.dicts("use_supplier",
    ((c, s) for c in components for s in suppliers),
    cat='Binary')

buffer = pulp.LpVariable.dicts("buffer",
    components,
    lowBound=0)

# Objective function: Minimize risk-adjusted cost
prob += (
    # Direct procurement costs
    pulp.lpSum(cost[c][s] * order[c,s] for c in components for s in suppliers) +
    # Risk premium based on reliability and geographic risk
    pulp.lpSum(cost[c][s] * order[c,s] * (1 - reliability[s]) * (1 + geo_risk[s])
               for c in components for s in suppliers) +
    # Buffer inventory costs
    pulp.lpSum(cost[c]['Supplier_1'] * 0.1 * buffer[c] for c in components)
)

# Add constraints here...
# 1. Meet demand with buffer
# 2. Supplier diversity requirements
# 3. Geographic risk balancing
# 4. Service level requirements
# 5. Supplier capacity limits

# Solve the model
prob.solve()

print(f"Status: {pulp.LpStatus[prob.status]}")
print(f"Optimal Risk-Adjusted Cost: ${pulp.value(prob.objective):,.2f}")